In [6]:
import sqlite3
import pandas as pd

# Path to your .db file
db_path = "CKD_train.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Step 1: List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
table_list = [t[0] for t in tables]  # Extract table names
print("Tables in database:", table_list)

# Step 2: Check structure of the 'labs' table
table_name = "labs"  # Ensure this is correct
if table_name in table_list:
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    print(f"Columns in {table_name}:", [(col[1], col[2]) for col in columns])
else:
    print(f"Table '{table_name}' not found! Check available tables.")
    conn.close()
    exit()

# Step 3: Retrieve lab values for a specific patient by ID
patient_id = 97360  # Ensure this ID exists
query = f"SELECT * FROM {table_name} WHERE Patient = ?"

try:
    df = pd.read_sql_query(query, conn, params=(str(patient_id),))  # Ensure ID is a string if needed
    if df.empty:
        print(f"No lab values found for Patient ID {patient_id}.")
    else:
        print(df)  # Display first few rows
        df.to_csv("patient_lab_values.csv", index=False)  # Save for inspection
        print("Patient data saved to 'patient_lab_values.csv'.")
except Exception as e:
    print("Error executing query:", e)

# Close the connection
conn.close()


Tables in database: ['diagnoses', 'medications', 'transplantations', 'labs', 'reports']
Columns in labs: [('Patient', 'INTEGER'), ('EntryDate', 'TEXT'), ('Analyte', 'TEXT'), ('ValueNumber', 'REAL'), ('ValueText', 'TEXT'), ('Unit', 'TEXT')]
    Patient   EntryDate  Analyte  ValueNumber ValueText      Unit
0     97360  2016-08-06  CKD-EPI        1.780      None  ml/s/spt
1     97360  2016-08-08  CKD-EPI        1.770      None  ml/s/spt
2     97360  2016-08-12  CKD-EPI        1.720      None  ml/s/spt
3     97360  2016-08-13  CKD-EPI        1.750      None  ml/s/spt
4     97360  2016-08-15  CKD-EPI        1.800      None  ml/s/spt
..      ...         ...      ...          ...       ...       ...
60    97360  2019-08-19     UPCR      500.760      None     g/mol
61    97360  2020-09-02     UPCR      674.600      None     g/mol
62    97360  2018-11-30       PU        1.340      None       g/l
63    97360  2019-08-19       PU        0.656      None       g/l
64    97360  2020-09-02       PU  

In [20]:
import sqlite3

conn = sqlite3.connect('CKD_train_unlocked.db')
cursor = conn.cursor()

# Enable WAL mode to prevent locking issues
cursor.execute("PRAGMA journal_mode=WAL;")
conn.commit()
conn.close()


In [21]:
import sqlite3

# Connect to the CKD database with WAL mode and timeout
conn = sqlite3.connect('CKD_train_unlocked.db', timeout=30)
cursor = conn.cursor()

# Enable WAL mode to avoid database lock issues
cursor.execute("PRAGMA journal_mode=WAL;")

# 1. Standardize simple formatting variations
cursor.execute("""
    UPDATE labs
    SET Unit = 'µmol/L'
    WHERE Unit IN ('umol/L', 'umol/l', 'µmol/l', 'μmol/L', 'μmol/l')
""")

cursor.execute("""
    UPDATE labs
    SET Unit = 'mmol/L'
    WHERE Unit IN ('mmol/l', 'MMOL/L')
""")

cursor.execute("""
    UPDATE labs
    SET Unit = 'mg/dL'
    WHERE Unit IN ('mg/dl', 'MG/DL', 'mg%')
""")

cursor.execute("""
    UPDATE labs
    SET Unit = 'g/dL'
    WHERE Unit IN ('g/dl', 'G/DL', 'g%')
""")

cursor.execute("""
    UPDATE labs
    SET Unit = 'mg/mmol'
    WHERE Unit IN ('mg/mMol', 'mg/mmole', 'mg per mmol')
""")

cursor.execute("""
    UPDATE labs
    SET Unit = 'mg/g'
    WHERE Unit IN ('mg/gram', 'mg per g', 'mg/gm')
""")

cursor.execute("""
    UPDATE labs
    SET Unit = 'mL/min/1.73 m²'
    WHERE Unit IN (
        'ml/min/1.73m2', 'mL/min/1.73m2', 'mL/min/1.73 M2', 
        'ml/min/1.73 m2', 'ml/min per 1.73m2'
    )
""")

# 2. Perform unit conversions
cursor.execute("""
    UPDATE labs
    SET ValueNumber = ValueNumber / 88.4, Unit = 'mg/dL'
    WHERE Unit = 'µmol/L' AND Analyte = 'Serum Creatinine'
""")

cursor.execute("""
    UPDATE labs
    SET ValueNumber = ValueNumber / 10.0, Unit = 'g/dL'
    WHERE Unit = 'g/L'
""")

cursor.execute("""
    UPDATE labs
    SET ValueNumber = ValueNumber / 10.0, Unit = 'mg/dL'
    WHERE Unit = 'mg/L'
""")

cursor.execute("""
    UPDATE labs
    SET ValueNumber = ValueNumber * 8.84, Unit = 'mg/g'
    WHERE Unit = 'mg/mmol' AND Analyte = 'Urine Albumin/Creatinine Ratio'
""")

# 3. Remove or nullify entries with unsupported units
cursor.execute("""
    DELETE FROM labs
    WHERE Unit IN (
        'mg/24h', 'mg/day', 'IU/L', 'U/L', '', 'none', 'N/A', 'ml/min'
    )
""")

# Commit changes and close connection
conn.commit()
conn.close()


In [2]:
import sqlite3
import pandas as pd

# Establish a connection to the database
connection = sqlite3.connect('CKD_train.db')  # Replace 'your_database.db' with your actual database file
cursor = connection.cursor()

# Define the query
query = """
SELECT *
FROM labs
WHERE unit IS NOT NULL;
"""

# Execute the query using the connection
cursor.execute(query)

# Fetch all results from the executed query
results = cursor.fetchall()

# Convert the results to a pandas DataFrame
columns = [desc[0] for desc in cursor.description]  # Get column names from the cursor description
df = pd.DataFrame(results, columns=columns)

# Display the number of records found
print(len(results), "records found.")

# Close the connection
connection.close()

# Display unique values of units
unique_units = df['Unit'].value_counts()
print(unique_units)

# Filter the DataFrame to keep only rows with the filtered units
#filtered_df = df[df['unit'].isin(filtered_units.index)]

# Display the cleaned DataFrame
#print(filtered_df)

241125 records found.
Unit
µmol/l             66835
ml/s/1,73 m2       66531
umol/l             64749
ml/s/spt           25271
g/mol              10901
g/l                 6047
ml/s/m2              215
umol/L               152
ml/s/1,73m2          106
µmol/L                64
mg/mmol               53
ml/s                  44
ml/s/kor              37
ml/s na 1,73 m2       31
ml/s/1,73 m^2         16
ml/s/1,73m^2          16
g/24h                 16
ml/s /1,73 m^2        15
ml/min/1,73m^2         9
ml/s . 1,73/m2         6
mmol/l                 3
g/L                    2
mg/mmolKr              2
g/mol kre              1
g/mmol                 1
ml/min/1,73 m^2        1
g/molkre               1
Name: count, dtype: int64


In [ ]:
import sqlite3

# Connect to the CKD database
conn = sqlite3.connect('CKD_train_unlocked.db')
cursor = conn.cursor()

# 1️⃣ Convert ml/s to mL/min (multiplying by 60)
cursor.execute("""
    UPDATE labs
    SET ValueNumber = ValueNumber * 60, Unit = 'mL/min/1.73 m²'
    WHERE Unit IN (
        'ml/s/1,73 m^2', 'ml/s/1,73m^2'
    )
""")

# 2️⃣ Standardize ml/min units
cursor.execute("""
    UPDATE labs
    SET Unit = 'mL/min/1.73 m²'
    WHERE Unit IN ('ml/min/1,73m^2', 'ml/min/1,73 m^2')
""")

# 3️⃣ Convert g/L and mg/mmol to g/mol
cursor.execute("""
    UPDATE labs
    SET Unit = 'g/mol'
    WHERE Unit = 'g/L'
""")

cursor.execute("""
    UPDATE labs
    SET ValueNumber = ValueNumber / 1000, Unit = 'g/mol'
    WHERE Unit = 'mg/mmol'
""")

# 4️⃣ Remove unclear units
cursor.execute("""
    DELETE FROM labs
    WHERE Unit IN ('ml/s/spt', 'g/24h')
""")

# Commit changes and close connection
conn.commit()
conn.close()

In [7]:
import sqlite3
import pandas as pd

# Establish a connection to the database
connection = sqlite3.connect('CKD_train_unlocked.db')  # Replace 'your_database.db' with your actual database file
cursor = connection.cursor()

# Define the query
query = """
SELECT *
FROM labs
WHERE unit IS NOT NULL;
"""

# Execute the query using the connection
cursor.execute(query)

# Fetch all results from the executed query
results = cursor.fetchall()

# Convert the results to a pandas DataFrame
columns = [desc[0] for desc in cursor.description]  # Get column names from the cursor description
df = pd.DataFrame(results, columns=columns)

# Display the number of records found
print(len(results), "records found.")

# Close the connection
connection.close()

# Display unique values of units
unique_units = df['Unit'].value_counts()
print(unique_units)

# Filter the DataFrame to keep only rows with the filtered units
#filtered_df = df[df['unit'].isin(filtered_units.index)]

# Display the cleaned DataFrame
#print(filtered_df)

215793 records found.
Unit
µmol/L            131800
mL/min/1.73 m²     66983
g/mol              10908
g/l                 6047
mg/mmol               55
Name: count, dtype: int64


In [2]:
import pandas as pd
import sqlite3

# Load CSV
csv_file = "patients.csv"
df = pd.read_csv(csv_file)

# Drop the "Unnamed: 0" column if it exists
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Connect to SQLite
db_file = "CKD_train_unlocked.db"
conn = sqlite3.connect(db_file)

# Overwrite table with cleaned data
df.to_sql("patients", conn, if_exists="replace", index=False)  

conn.commit()
conn.close()

print("Table updated without 'Unnamed: 0' column.")

Table updated without 'Unnamed: 0' column.
